In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasci_tools

In [3]:
from os import sys
sys.path.append("/neurd_packages/meshAfterParty/meshAfterParty/")
sys.path.append("/neurd_packages/machine_learning_tools/machine_learning_tools/")
sys.path.append("/neurd_packages/neuron_morphology_tools/neuron_morphology_tools/")

In [4]:
import datajoint_utils as du
import trimesh_utils as tu

WARNING - 2023-06-20 04:24:26,590 - _default - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


Using default Google credentials. There is no ~/.cloudvolume/secrets/google-secret.json set.


WARNING - 2023-06-20 04:24:28,350 - utils - Imported datajoint_plus version, 0.1.0 does not match the latest version on Github, 0.1.7.
INFO - 2023-06-20 04:24:28,356 - settings - Setting enable_python_native_blobs to True
INFO - 2023-06-20 04:24:28,358 - settings - Setting stores to {'h01_skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/microns/h01/skeletons', 'stage': '/mnt/dj-stor01/microns/h01/skeletons'}, 'h01_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/microns/h01/meshes', 'stage': '/mnt/dj-stor01/microns/h01/meshes'}, 'h01_faces': {'protocol': 'file', 'location': '/mnt/dj-stor01/microns/h01/faces', 'stage': '/mnt/dj-stor01/microns/h01/faces'}, 'h01_decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/microns/h01/decimated_meshes', 'stage': '/mnt/dj-stor01/microns/h01/decimated_meshes'}, 'h01_soma_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/microns/h01/soma_meshes', 'stage': '/mnt/dj-stor01/microns/h01/soma_meshes'}}
INFO - 20

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2023-06-20 04:24:30,107 - settings - Setting enable_python_native_blobs to True
INFO - 2023-06-20 04:24:39,134 - settings - Setting enable_python_native_blobs to True
ERROR - 2023-06-20 04:24:39,170 - compatibility - Could not add DataJointPlus to ImportMethod.
Traceback (most recent call last):
  File "/datajoint-plus/datajoint_plus/compatibility.py", line 41, in add_datajoint_plus
    obj.parse_hash_info_from_header()
  File "/datajoint-plus/datajoint_plus/base.py", line 354, in parse_hash_info_from_header
    if result[0] == attr:
IndexError: list index out of range
ERROR - 2023-06-20 04:24:39,231 - compatibility - Could not add DataJointPlus to Materialization.
Traceback (most recent call last):
  File "/datajoint-plus/datajoint_plus/compatibility.py", line 41, in add_datajoint_plus
    obj.parse_hash_info_from_header()
  File "/datajoint-plus/datajoint_plus/base.py", line 354, in parse_hash_info_from_header
    if result[0] == attr:
IndexError: list index out of range
ERROR

Output()

INFO - 2023-06-20 04:24:50,038 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2023-06-20 04:24:50,039 - settings - Setting database.user to celiib
INFO - 2023-06-20 04:24:50,040 - settings - Setting database.password to newceliipass
INFO - 2023-06-20 04:24:50,042 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2023-06-20 04:24:50,043 - settings - Setting database.user to celiib
INFO - 2023-06-20 04:24:50,044 - settings - Setting database.password to newceliipass
INFO - 2023-06-20 04:24:50,045 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2023-06-20 04:24:50,046 - settings - Setting database.user to celiib
INFO - 2023-06-20 04:24:50,047 - settings - Setting database.password to newceliipass
INFO - 2023-06-20 04:24:50,054 - settings - Setting enable_python_native_blobs to True
INFO - 2023-06-20 04:24:50,840 - settings - Setting enable_python_native_blobs to True
INFO - 2023-06-20 04:24:50,919 - settings - Setting database.hos

# Importing the Data

In [5]:
import trimesh

In [8]:
good_mesh = trimesh.load_mesh(
    file_obj = "./good_mesh.obj"
)

unsmooth = trimesh.load_mesh(
    file_obj = "./unsmooth_mesh.obj"
)

In [9]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_mesh(good_mesh)

Mesh(color=array('green', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [10]:
ipvu.plot_mesh(unsmooth)

Mesh(color=array('green', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

# Mesh processing

In [19]:
"""
Pseudocode: 
1) Poisson surface reconstruction
2) Find number of connected components
"""
largest_conn_mesh = tu.largest_conn_comp(
    unsmooth,
    connectivity="edges",
)

poisson_mesh = tu.poisson_surface_reconstruction(
    largest_conn_mesh,
)

curr_mesh = poisson_mesh
ipvu.plot_mesh(curr_mesh)

xvfb-run -n 5570 -s "-screen 0 800x600x24" meshlabserver $@  -i /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/mesh_459.off -o /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/mesh_459_poisson.off -s /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/poisson_774156.mls
removed temporary input file: /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/mesh_459.off
removed temporary output file: /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/mesh_459_poisson.off


Mesh(color=array('green', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [32]:
stitch_mesh = tu.decimate(largest_conn_mesh,0.10)
stitch_mesh = tu.fill_holes(stitch_mesh)
stitch_mesh=  tu.poisson_surface_reconstruction(stitch_mesh)
stitch_mesh = tu.largest_conn_comp(stitch_mesh)
ipvu.plot_mesh(stitch_mesh)

xvfb-run -n 4522 -s "-screen 0 800x600x24" meshlabserver $@  -i /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/neuron_90335.off -o /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/neuron_90335_decimated.off -s /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/decimation_meshlab_10851099.mls
xvfb-run -n 8923 -s "-screen 0 800x600x24" meshlabserver $@  -i /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/neuron_88358.off -o /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/neuron_88358_fill_holes.off -s /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/fill_holes_936968.mls
removed temporary input file: /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/neuron_88358.off
removed temporary output file: /neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp/neuron_88358_fill_holes.off
/neurd_packages/meshAfterParty/Applications/mesh_preprocessing/temp

/neurd_packages/meshAfterParty/meshAfterParty/trimesh_utils.py:3595: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



Mesh(color=array('green', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [33]:
ipvu.save_to_html("./unsmoothed_processed.html")